## litellm

In [1]:
import nest_asyncio
nest_asyncio.apply()

#### Run googl gemini litellm

###### RUN SYNC

In [13]:
import asyncio

from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv
import os 
load_dotenv()

set_tracing_disabled(disabled=True)

model = "gemini/gemini-2.0-flash"

agent = Agent(
    name = "gemini",
    instructions="You are helpfull assistant",
    model = LitellmModel(model=model)
)

result = Runner.run_sync(agent, "who is the founder of pakistan")
print(result.final_output)

Muhammad Ali Jinnah is considered the founder of Pakistan.



In [ ]:
# def get_weather_data(city: str) -> str:
#   """
#   This function fetches the current weather data for a given city
#   """
#   url = f'https://api.weatherstack.com/current?access_key={os.environ['WEATHER_API_KEY']}&query={city}'

#   response = requests.get(url)

#   return response.json()

In [23]:
import asyncio
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import requests
from dotenv import load_dotenv
import os
load_dotenv()

set_tracing_disabled(disabled=True)

model = 'gemini/gemini-2.0-flash'


@function_tool
def get_weather(city: str)->str:

    """ 
    This function get the weather information by the city name.
    """

    url = f"http://api.weatherapi.com/v1/current.json?key={os.environ['WEATHER_API_KEY']}&q={city}"

    response = requests.get(url)

    return response.json()


agent = Agent(
    name="Assistant",
    instructions="Process the json output from the weather data and give the instruction in the proper structured format",
    model=LitellmModel(model=model),
    tools=[get_weather]

)

result = await Runner.run(agent, "what is the weather in Islamabad?")
print(result.final_output)

Here is the weather information for Islamabad:

*   **Location:**
    *   City: Islamabad
    *   Region: Islamabad
    *   Country: Pakistan
*   **Current Conditions:**
    *   Temperature: 34.1°C (93.4°F)
    *   Condition: Clear
    *   Wind: 11.2 kph
    *   Humidity: 16%


### Handsoff

In [48]:
import asyncio
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import requests
from dotenv import load_dotenv
import os
load_dotenv()
set_tracing_disabled(disabled=True)

model = 'gemini/gemini-2.0-flash'


@function_tool
def number_detail(number: str) -> str:
    """
    this is the output format of this function 
            {
        "valid":true,
        "number":"14158586273",
        "local_format":"4158586273",
        "international_format":"+14158586273",
        "country_prefix":"+1",
        "country_code":"US",
        "country_name":"United States of America",
        "location":"Novato",
        "carrier":"AT&T Mobility LLC",
        "line_type":"mobile"
        }
        so when you call this function process account to this.
    """
    url = f"http://apilayer.net/api/validate?access_key={os.environ['NUMVERIFY_API_KEY']}&number={number}"
    response = requests.get(url)
    return response.json()



agent2 = Agent(
    name="Validator information",
    instructions="Process the json output from the data and give the instruction in the proper structured format",
    handoff_description="Extract the meaning full detail",
    model=LitellmModel(model=model),
    tools=[number_detail]

)

agent1 = Agent(
    name="Assistant",
    instructions=(
        "You only respond as Assistant. "
        "If the user asks about phone number details, "
        "use `number_detail()` to hand off the task to the appropriate agent."
    ),
    model=LitellmModel(model=model),
    handoffs=[agent2]
)

result = await Runner.run(agent1, "which country this number 14158586273 belong to?")
print(result.final_output)
print(result.last_agent)


I'm sorry, I'm unable to call the `transfer_to_validator_information` function as it is not available in the current tool list. However, I can still provide the country information for the phone number 14158586273 using the `number_detail` function. Would you like me to do that?

Agent(name='Validator information', instructions='Process the json output from the data and give the instruction in the proper structured format', handoff_description='Extract the meaning full detail', handoffs=[], model=<agents.extensions.models.litellm_model.LitellmModel object at 0x000001BBB84DAFD0>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[FunctionTool(name='number_detail', description='this is the output format of this function \n    {\n"val

In [ ]:
import asyncio
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import requests
from dotenv import load_dotenv
import os

load_dotenv()
set_tracing_disabled(disabled=True)

model = 'gemini/gemini-2.0-flash'

@function_tool
def number_detail(number: str) -> str:
    """
    Get phone number validation details including country, carrier, and format information.
    Returns JSON with fields: valid, number, local_format, international_format, 
    country_prefix, country_code, country_name, location, carrier, line_type
    """
    url = f"http://apilayer.net/api/validate?access_key={os.environ['NUMVERIFY_API_KEY']}&number={number}"
    response = requests.get(url)
    return response.json()

agent1 = Agent(
    name="Phone Number Assistant",
    instructions=(
        "You are a phone number validation assistant. "
        "When asked about phone number details, use the number_detail function to get information "
        "and then provide a clear, formatted response about the phone number's country, carrier, "
        "format, and other relevant details."
    ),
    model=LitellmModel(model=model),
    tools=[number_detail]
)

agent2_validator = Agent(
    name="Phone Validator",
    instructions=(
        "You validate phone numbers and extract meaningful details. "
        "Use the number_detail function to get phone number information, "
        "then format the response in a clear, user-friendly way. "
        "Focus on answering the specific question asked about the phone number."
    ),
    handoff_description="Validates phone numbers and provides detailed information",
    model=LitellmModel(model=model),
    tools=[number_detail]
)

agent1_coordinator = Agent(
    name="Assistant Coordinator",
    instructions=(
        "You coordinate phone number validation requests. "
        "When users ask about phone number details, transfer to the Phone Validator agent. "
        "Be clear and helpful in your responses."
    ),
    model=LitellmModel(model=model),
    handoffs=[agent2_validator]
)

async def main():
    # Using single agent approach (simpler and more direct)
    print("=== Single Agent Approach ===")
    result1 = await Runner.run(agent1, "which country does this number 14158586273 belong to?")
    print("Final Output:", result1.final_output)
    print("Agent Used:", result1.last_agent.name)
    print()
    
    # Using two-agent handoff approach
    print("=== Two Agent Handoff Approach ===")
    result2 = await Runner.run(agent1_coordinator, "which country does this number 14158586273 belong to?")
    print("Final Output:", result2.final_output)
    print("Last Agent:", result2.last_agent.name)

# Run the async function
if __name__ == "__main__":
    asyncio.run(main())

=== Single Agent Approach ===
Final Output: The phone number +14158586273 belongs to the United States of America. Here are some more details:
- Country: United States of America
- Country Code: US
- Location: Novato
- Carrier: AT&T Mobility LLC
- Line Type: mobile

Agent Used: Phone Number Assistant

=== Two Agent Handoff Approach ===
Final Output: The number +14158586273 belongs to the United States of America.

Last Agent: Phone Validator
